<a href="https://colab.research.google.com/github/Vinod181714/Employee-Attrition-ML/blob/main/Notification_Prioritization_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notification Prioritization Engine
Author: Vinod S R  
Platform: Google Colab  

This notebook implements a notification decision engine that classifies
each notification as NOW / LATER / NEVER with explainable reasoning.

In [ ]:
import datetime
import hashlib
from collections import defaultdict, deque
import random
import numpy as np

In [ ]:
RULE_CONFIG = {
    "urgency_weight": 0.4,
    "impact_weight": 0.3,
    "recency_weight": 0.2,
    "behavior_weight": 0.1,

    "NOW_THRESHOLD": 7.0,
    "LATER_THRESHOLD": 4.0,

    "MAX_ALERTS_PER_HOUR": 3,
    "DEDUP_WINDOW_MINUTES": 10,
    "RECENCY_DECAY_MINUTES": 60
}

In [ ]:
class NotificationEvent:
    def __init__(self, event_id, user_id, event_type, message,
                 source, urgency, business_impact,
                 channel, timestamp, expires_at=None, dedupe_key=None):

        self.event_id = event_id
        self.user_id = user_id
        self.event_type = event_type
        self.message = message
        self.source = source
        self.urgency = urgency              # 1–3
        self.business_impact = business_impact  # 1–5
        self.channel = channel
        self.timestamp = timestamp
        self.expires_at = expires_at
        self.dedupe_key = dedupe_key
        self.content_hash = self._hash()

    def _hash(self):
        base = self.dedupe_key if self.dedupe_key else self.message
        return hashlib.md5(base.encode()).hexdigest()


class UserContext:
    def __init__(self, user_id, response_rate):
        self.user_id = user_id
        self.response_rate = response_rate

In [ ]:
from collections import defaultdict, deque

user_notification_history = defaultdict(deque)   # timestamps
recent_dedupe_cache = {}                          # (user, hash) → time
decision_audit_log = []                           # explainability

In [ ]:
def is_duplicate(notification):
    key = (notification.user_id, notification.content_hash)
    now = datetime.datetime.now()

    if key in recent_dedupe_cache:
        delta = (now - recent_dedupe_cache[key]).seconds / 60
        if delta < RULE_CONFIG["DEDUP_WINDOW_MINUTES"]:
            return True

    recent_dedupe_cache[key] = now
    return False

In [ ]:
def exceeds_rate_limit(notification):
    history = user_notification_history[notification.user_id]
    now = datetime.datetime.now()

    # Remove old entries (> 1 hour)
    while history and (now - history[0]).seconds > 3600:
        history.popleft()

    return len(history) >= RULE_CONFIG["MAX_ALERTS_PER_HOUR"]

In [ ]:
def compute_priority(notification, user):
    age_minutes = (datetime.datetime.now() - notification.timestamp).seconds / 60
    recency_score = max(0, 1 - age_minutes / RULE_CONFIG["RECENCY_DECAY_MINUTES"])

    score = (
        RULE_CONFIG["urgency_weight"] * notification.urgency +
        RULE_CONFIG["impact_weight"] * notification.business_impact +
        RULE_CONFIG["recency_weight"] * recency_score +
        RULE_CONFIG["behavior_weight"] * user.response_rate
    )

    return round(score * 2, 2)

In [ ]:
def predict_response_probability(notification, user):
    try:
        return round(
            0.5 * user.response_rate +
            0.5 * (notification.urgency / 3),
            2
        )
    except:
        return user.response_rate

In [ ]:
def classify_decision(priority_score, notification):
    if notification.expires_at and datetime.datetime.now() > notification.expires_at:
        return "NEVER"

    if priority_score >= RULE_CONFIG["NOW_THRESHOLD"]:
        return "NOW"
    elif priority_score >= RULE_CONFIG["LATER_THRESHOLD"]:
        return "LATER"
    else:
        return "NEVER"

In [ ]:
def process_notification(notification, user):
    reasons = []

    # 1. Deduplication
    if is_duplicate(notification):
        decision = "NEVER"
        reasons.append("duplicate notification")
        return decision, reasons

    # 2. Rate limiting
    if exceeds_rate_limit(notification):
        decision = "LATER"
        reasons.append("rate limit exceeded")

    # 3. Priority scoring
    base_score = compute_priority(notification, user)
    ml_score = predict_response_probability(notification, user)
    final_score = base_score + ml_score

    # 4. Final decision
    decision = classify_decision(final_score, notification)

    # 5. Reason logging
    reasons.extend([
        f"urgency={notification.urgency}",
        f"impact={notification.business_impact}",
        f"priority_score={final_score}"
    ])

    # 6. Persist history if sent
    if decision == "NOW":
        user_notification_history[notification.user_id].append(
            datetime.datetime.now()
        )

    # 7. Audit log
    decision_audit_log.append({
        "event_id": notification.event_id,
        "user_id": notification.user_id,
        "decision": decision,
        "reasons": reasons
    })

    return decision, reasons

In [ ]:
user = UserContext("U1", response_rate=0.7)

events = [
    NotificationEvent(
        "E1", "U1", "lead", "New hot lead",
        "CRM", urgency=3, business_impact=5,
        channel="push",
        timestamp=datetime.datetime.now()
    ),
    NotificationEvent(
        "E2", "U1", "promo", "Buy now!",
        "Marketing", urgency=1, business_impact=1,
        channel="email",
        timestamp=datetime.datetime.now()
    ),
    NotificationEvent(
        "E3", "U1", "lead", "New hot lead",
        "CRM", urgency=3, business_impact=5,
        channel="push",
        timestamp=datetime.datetime.now()
    )
]

for e in events:
    decision, reasons = process_notification(e, user)
    print(e.event_id, "→", decision, "|", reasons)

E1 → LATER | ['urgency=3', 'impact=5', 'priority_score=6.79']
E2 → NEVER | ['urgency=1', 'impact=1', 'priority_score=2.46']
E3 → NEVER | ['duplicate notification']


In [ ]:
for log in decision_audit_log:
    print(log)

{'event_id': 'E1', 'user_id': 'U1', 'decision': 'LATER', 'reasons': ['urgency=3', 'impact=5', 'priority_score=6.79']}
{'event_id': 'E2', 'user_id': 'U1', 'decision': 'NEVER', 'reasons': ['urgency=1', 'impact=1', 'priority_score=2.46']}
